First we need all the libraries 

numpy,pandas & pylab for signal working

tkinter for interface build up

In [23]:
#First we need all the libraries 
#numpy,pandas & pylab for signal working
#tkinter for interface build up
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from PIL import ImageTk,Image
import numpy as np
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
import sys

#Initialize the window for data giving
window = tk.Tk()
window.minsize(500,500)
window.title('Automated Robust First Peak Detection')

stop_called = False

#All the Functions will be here

#Main algorithm to calculate z-score finding the range of peaks
def z_score_algo(dataArray, lag, threshold, influence):
    '''
    z-score algorithm
    To initialize this function the data has to be converted into a pandas array
    lag is to determine the smoothness and adaptiveness of the algorithm (from where the filter starts)
    threshold is to determine the least point from which new high value is counted (changes with signal pattern)
    influence is to select how much the current filter should rely on past learning of signal
    '''
    signals = np.zeros(len(dataArray))
    filteredData = np.array(dataArray)
    avgFilter = [0]*len(dataArray)
    stdFilter = [0]*len(dataArray)
    avgFilter[lag - 1] = np.mean(dataArray[0:lag])
    stdFilter[lag - 1] = np.std(dataArray[0:lag])
    for i in range(lag, len(dataArray)):
        if abs(dataArray[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if dataArray[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredData[i] = influence * dataArray[i] + (1 - influence) * filteredData[i-1]
            avgFilter[i] = np.mean(filteredData[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredData[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredData[i] = dataArray[i]
            avgFilter[i] = np.mean(filteredData[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredData[(i-lag+1):i+1])
    
    unitPositions = np.where(np.asarray(signals) == 1)

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter),
               unitPositions = unitPositions)

#Helping Functions
#This is the function for finding highest value from an array
def find_highest_value(array, numberOfHighestValues):
    sortedIndexes = np.argsort(array)
    sortedArray = array[sortedIndexes]
    highestValue = sortedArray[-numberOfHighestValues : ]
    return highestValue.tolist()

#This is the function for finding appropriate lag
def find_lag(array, noiseAmp):
    arr = np.where(array >= noiseAmp, 1, 0)
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    isZero = np.concatenate(([0], np.equal(arr, 0).view(np.int8), [0]))
    absDiff = np.abs(np.diff(isZero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absDiff == 1)[0].reshape(-1, 2)
    return ranges[0, 1] - 1

#This is the function for closing the windows
def on_closing():
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        window.destroy()

#This is the main processing function called uppon start button
def start_process():
    global stop_called
    stop_called = False
    
    print('start_process being called')
    try:
        print(file_path_text.get())
        #If the fields are not fulfilled throw an error message
        if file_path_text.get() == '' or sig_number_text.get() == '':
            messagebox.showerror('Requires fields', 'You have not filled all the required entries. * Marked fields are mendatory. Please give all the inputs properly')
            return
        
        #Wireup desired signal number
        signalNumber = int(sig_number_text.get())        
        
        #Put the default lag value as 0.15
        noiseAmp = None
        if lag_number_text.get() == '':
            noiseAmp = 0.15
        else:
            noiseAmp = float(lag_number_text.get())
            
        #Put the default influence value as 0.1
        influence = None
        if influence_number_text.get() == '':
            influence = 0.1
        else:
            influence = float(influence_number_text.get())
            
        #Determination of threshold as signal number
        thresholdValues = {'1': 25, '2': 25, '3': 25, '4': 25, '5': 25, '6': 25, '7': 25, '8': 25, '9': 25, '10': 25,
                           '11': 25, '12': 25, '13': 25, '14': 25, '15': 25, '16': 25, '17': 25, '18': 25, '19': 25, '20': 25,
                           '21': 25, '22': 25, '23': 25, '24': 25, '25': 25, '26': 25, '27': 25, '28': 25, '29': 25, '30': 25,
                           '31': 25, '32': 25, '33': 25, '34': 22, '35': 25, '36': 25, '37': 25, '38': 25, '39': 25, '40': 25,
                           '41': 25, '42': 25, '43': 25, '44': 25, '45': 25, '46': 25, '47': 25, '48': 25, '49': 25, '50': 25}
        threshold = None
        if threshold_number_text.get() == '':
            threshold = thresholdValues[str(signalNumber)]
        else:
            threshold = int(threshold_number_text.get())
        
       
        
        #Now we need to load the file as a data frame object
        df = pd.read_excel(file_path_text.get(), header=None)
        columnName = []
        for i in range(len(df.axes[1])):
            name = "col" + str(i)
            columnName.append(name)
        df.columns = columnName
        
        #Here discarding of first 6 columns will occur as they were not part of the signal in the original data file
        df = df.drop(columns=["col0", "col1", "col2", "col3", "col4", "col5"])
        
        #Find the desired signal
        y = df.iloc[signalNumber - 1]
        copyOfY = np.copy(y)
        
        highestPeaks = []
        for i in range(len(df)):
            b = df.iloc[i]
            highestValue = find_highest_value(b, 1)
            highestPeaks.extend(highestValue)
        
        #As the data signals are smoothed with a very little difference between values and peaks we make the span bigger
        y = np.where((y >= 0.1) & (y <= 0.2), 0.001, y)
        y = np.where((y >= 0.2) & (y <= 0.3), 0.005, y)
        y = np.where((y >= 0.3) & (y <= 0.31), 1, y)
        y = np.where((y >= 0.31) & (y <= 0.32), 10, y)
        y = np.where((y >= 0.32) & (y <= 0.321), 100, y)
        y = np.where( (y > 0.321) & (y <= 0.322), 1000, y)
        y = np.where( (y > 0.322) & (y <= 0.323), 10000, y)
        y = np.where( (y > 0.323) & (y <= 0.324), 100000, y)
        y = np.where( (y > 0.324) & (y <= 0.325), 1000000, y)
        y = np.where( (y > 0.325) & (y <= 0.326), 10000000, y)
        y = np.where( (y> 0.326), 100000000, y)
        
        lag = find_lag(y, noiseAmp)
        
        result = z_score_algo(y, lag= lag, threshold= threshold, influence= influence)
        unitPositionValues = np.take(copyOfY, result["unitPositions"])
        peak = find_highest_value(unitPositionValues[0], 1)
        
        resultSentence = ("The peak lies between point " + str(result["unitPositions"][0][0]) +
                          " and " + str(result["unitPositions"][0][-1]) + "\n" +
                          "The peak value is " + str(peak[0]) + "\n" +
                         "The exact peak point is at " + str(np.where(copyOfY == peak[0])[0][0]))
        result_showing_label.config(text=resultSentence)
        result_showing_label.update()
        
        #plotting part
        fig1 = Figure(figsize= (5,5), dpi= 100)
        mainSig = fig1.add_subplot(211)
        mainSig.plot(np.arange(0, len(copyOfY)), copyOfY)
        chart1 = FigureCanvasTkAgg(fig1, window)
        chart1.get_tk_widget().grid(row=7, column=0)
        
        filteredSig = fig1.add_subplot(212).plot(np.arange(0, len(y)), result["avgFilter"], color='cyan', lw= 2)
        #fig.plot(np.arange(0, len(y)), result["avgFilter"] + threshold*result["stdFilter"], color= "green", lw= 2)
        
        fig2 = Figure(figsize= (5,5), dpi= 100)
        peakDetectionPoints = fig2.add_subplot(111)
        peakDetectionPoints.plot(np.arange(0, len(y)), result["signals"], color= "red", lw= 2)
        chart2 = FigureCanvasTkAgg(fig2, window)
        chart2.get_tk_widget().grid(row=7, column=1)
        
    except IndexError:
        pass
    start_button.config(state=tk.ACTIVE)
    

#This section holds the User Interface building

#File Path
file_path_text = tk.StringVar(window)
file_path_label = ttk.Label(window, text='Please give the data file path*: ', font=('bold', 10), padding=10)
file_path_label.grid(row=0, column=0, sticky=tk.W)
file_path_entry = ttk.Entry(window, textvariable=file_path_text, width=100)
file_path_entry.grid(row=0, column=1)

#Signal Number
sig_number_text = tk.StringVar()
sig_number_label = ttk.Label(window, text='Please give the desired signal number(1-50)*: ', font=('bold', 10), padding=10)
sig_number_label.grid(row=1, column=0, sticky=tk.W)
sig_number_entry = ttk.Entry(window, textvariable=sig_number_text, width=10)
sig_number_entry.grid(row=1, column=1)

#Optional NoiseCuttingAmp
lag_number_text = tk.StringVar()
lag_number_label = ttk.Label(window, text='Give the Noise Ignoring Amplitude value 0.1-0.3 (optional, default is 0.15): ', font=('bold', 10), padding=10)
lag_number_label.grid(row=2, column=0, sticky=tk.W)
lag_number_entry = ttk.Entry(window, textvariable=lag_number_text, width=10)
lag_number_entry.grid(row=2, column=1)

#Optional Define Threshold
threshold_number_text = tk.StringVar()
threshold_label = ttk.Label(window, text='Give the initial threshold value 1-100 (optional, default is selected from signal criteria): ', font=('bold', 10), padding=10)
threshold_label.grid(row=3, column=0, sticky=tk.W)
threshold_number_entry = ttk.Entry(window, textvariable=threshold_number_text, width=10)
threshold_number_entry.grid(row=3, column=1)

#Optional Influence
influence_number_text = tk.StringVar()
influence_label = ttk.Label(window, text='Give the influence of previous signal values 0.0-1.0 (optional, default is 0.1): ', font=('bold', 10), padding=10)
influence_label.grid(row=4, column=0, sticky=tk.W)
influence_number_entry = ttk.Entry(window, textvariable=influence_number_text, width=10)
influence_number_entry.grid(row=4, column=1)

#Start Button
start_button = ttk.Button(window, text="Start", state=tk.NORMAL, command=start_process)
start_button.grid(row=5, column=1)

#Result Showing Label
result_showing_label = ttk.Label(window, text='Result will be here', font=('bold', 20), width=50, background='#ffffff')
result_showing_label.grid(row=6, column=0)

window.protocol("WM_DELETE_WINDOW", on_closing)
window.mainloop()

start_process being called
DataSetFile.xlsx
